In [1]:
import numpy as np
from tqdm import tqdm
import keras
from keras import Model
from keras.preprocessing import timeseries_dataset_from_array

import modules.blocks as blocks
import modules.data as data
from modules.utilites import DataClass

In [2]:
# Model parametrs
class Input_pars(DataClass):
    seq_len = 100


class CN_pars(DataClass):
    n_filters = 14
    dilation_steps = 4  # dilation = 2**dilation_step


class AN_pars(DataClass):
    attention_heads = 3
    blocks = 2
    share_weights = True


class FF_pars(DataClass):
    dropout_rate = 0.1


class Optimaser_pars(DataClass):
    lr = 0.0001
    adam_beta1 = 0.9
    adam_beta2 = 0.999


class Trainig_pars(DataClass):
    shuffle = True
    batch_size = 512
    epochs = 150


class Full_pars(DataClass):
    seq_len = 100
    cn = CN_pars()
    an = AN_pars()
    ff = FF_pars()
    optimazer = Optimaser_pars()
    training = Trainig_pars()


pars = Full_pars()

In [3]:
(x_train, y_train), (x_val, y_val), (x_test,y_test) = data.load_dataset(horizon=4) # yapf:disabele

In [4]:
ds_train = data.build_dataset(
    
    x=x_train,
    y=y_train,
    batch_size=pars.training.batch_size,
    seq_len=pars.seq_len,
)

ds_val = data.build_dataset(
    x=x_val,
    y=y_val,
    batch_size=pars.training.batch_size,
    seq_len=pars.seq_len,
)

ds_test = data.build_dataset(
    x=x_test,
    y=y_test,
    batch_size=pars.training.batch_size,
    seq_len=pars.seq_len,
)

In [5]:
# Model
inputs = blocks.input_block(pars.seq_len)
# cnn = blocks.cnn_block(
#     input_layer=inputs,
#     filters=pars.cn.n_filters,
#     dilation_steps=pars.cn.dilation_steps,
# )
# norm = blocks.norm_block(input_layer=cnn)
# pos = blocks.positional_encoder_block(input_layer=norm)
transformer = blocks.transformer_block(
    input_layer=inputs,
    n_blocks=pars.an.blocks,
    n_heads=pars.an.attention_heads,
)
ffn = blocks.ffn_block(
    input_layer=transformer,
    dropout_rate=pars.ff.dropout_rate,
)
model = Model(inputs=inputs, outputs=ffn)

ValueError: Exception encountered when calling layer "private__transformer_block" (type _TransformerBlock).

in user code:

    File "d:\WORKS\translob\LOB\modules\blocks.py", line 384, in call  *
        output = self.attention_layer(x)
    File "d:\WORKS\translob\translob_env\Lib\site-packages\keras\src\utils\traceback_utils.py", line 70, in error_handler  **
        raise e.with_traceback(filtered_tb) from None
    File "d:\WORKS\translob\LOB\modules\blocks.py", line 102, in build
        self.validate_model_dimensionality(d_model)
    File "d:\WORKS\translob\LOB\modules\blocks.py", line 147, in validate_model_dimensionality
        raise ValueError(

    ValueError: The size of the last dimension of the input (40) must be evenly divisible by the numberof the attention heads 3


Call arguments received by layer "private__transformer_block" (type _TransformerBlock):
  • x=tf.Tensor(shape=(None, 100, 40), dtype=float32)
  • kwargs={'training': 'None'}

In [10]:
# CompiLe

model.summary()

model.compile(
    keras.optimizers.Adam(
        learning_rate=pars.optimazer.lr,
        beta_1=pars.optimazer.adam_beta1,
        beta_2=pars.optimazer.adam_beta2,
        name="Adam",
    ),
    loss=keras.losses.SparseCategoricalCrossentropy(),
    metrics=['sparse_categorical_accuracy'],
)
print(
    f'Train x: {str(x_train.shape): <15} - y: {y_train.shape}',
    f'Val   x: {str(x_val.shape): <15} - y: {y_val.shape}',
    sep='\n',
)

Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_3 (InputLayer)        [(None, 100, 40)]         0         
                                                                 
 conv1d_2 (Conv1D)           (None, 100, 14)           1134      
                                                                 
 layer_normalization_2 (Lay  (None, 100, 14)           28        
 erNormalization)                                                
                                                                 
 private__positional_encodi  (None, 100, 15)           0         
 ng_layer_2 (_PositionalEnc                                      
 odingLayer)                                                     
                                                                 
Total params: 1162 (4.54 KB)
Trainable params: 1162 (4.54 KB)
Non-trainable params: 0 (0.00 Byte)
_____________________________

In [ ]:
model.fit(
    ds_train,
    epochs=pars.training.epochs,
    validation_data=ds_val,
)
